In [1]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'
VERSION = 'HARDWARE'
SS_VER = 'SS_VER_2_1'
allowable_exceptions = None
CRYPTO_TARGET = 'TINYAES128C'



In [4]:
if VERSION == 'HARDWARE':

    #!/usr/bin/env python
    # coding: utf-8

    #
    #
    # **THIS IS NOT THE COMPLETE TUTORIAL - see file with (MAIN) in the name. Paste all this code before the first Python block**

    # First you'll need to select which hardware setup you have. You'll need to select both a `SCOPETYPE` and a `PLATFORM`. `SCOPETYPE` can either be `'OPENADC'` for the CWLite/CW1200 or `'CWNANO'` for the CWNano. `PLATFORM` is the target device, with `'CWLITEARM'`/`'CW308_STM32F3'` being the best supported option, followed by `'CWLITEXMEGA'`/`'CW308_XMEGA'`, then by `'CWNANO'`. As of CW 5.4, you can select the SimpleSerial version
    # used. For example:
    #
    # ```python
    # SCOPETYPE = 'OPENADC'
    # PLATFORM = 'CWLITEARM'
    # SS_VER = 'SS_VER_1_1'
    # ```

    # In[ ]:





    # This code will connect the scope and do some basic setup. We're now just going to use a special setup script to do this. This script contains the commands we ran seperately before.

    # In[ ]:



    #!/usr/bin/env python
    # coding: utf-8

    # In[ ]:


    import chipwhisperer as cw

    try:
        if not scope.connectStatus:
            scope.con()
    except NameError:
        scope = cw.scope(hw_location=(5, 16))

    try:
        if SS_VER == "SS_VER_2_1":
            target_type = cw.targets.SimpleSerial2
        elif SS_VER == "SS_VER_2_0":
            raise OSError("SS_VER_2_0 is deprecated. Use SS_VER_2_1")
        else:
            target_type = cw.targets.SimpleSerial
    except:
        SS_VER="SS_VER_1_1"
        target_type = cw.targets.SimpleSerial

    try:
        target = cw.target(scope, target_type)
    except:
        print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
        print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
        scope = cw.scope(hw_location=(5, 16))
        target = cw.target(scope, target_type)


    print("INFO: Found ChipWhisperer😍")


    # In[ ]:


    if "STM" in PLATFORM or PLATFORM == "CWLITEARM" or PLATFORM == "CWNANO":
        prog = cw.programmers.STM32FProgrammer
    elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        prog = cw.programmers.XMEGAProgrammer
    elif "neorv32" in PLATFORM.lower():
        prog = cw.programmers.NEORV32Programmer
    elif PLATFORM == "CW308_SAM4S":
        prog = cw.programmers.SAM4SProgrammer
    else:
        prog = None


    # In[ ]:


    import time
    time.sleep(0.05)
    scope.default_setup()

    if PLATFORM == "CW308_SAM4S":
        scope.io.target_pwr = 0
        time.sleep(0.2)
        scope.io.target_pwr = 1
        time.sleep(0.2)
    def reset_target(scope):
        if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
            scope.io.pdic = 'low'
            time.sleep(0.1)
            scope.io.pdic = 'high_z' #XMEGA doesn't like pdic driven high
            time.sleep(0.1) #xmega needs more startup time
        elif "neorv32" in PLATFORM.lower():
            raise IOError("Default iCE40 neorv32 build does not have external reset - reprogram device to reset")
        elif PLATFORM == "CW308_SAM4S":
            scope.io.nrst = 'low'
            time.sleep(0.25)
            scope.io.nrst = 'high_z'
            time.sleep(0.25)
        else:
            scope.io.nrst = 'low'
            time.sleep(0.05)
            scope.io.nrst = 'high_z'
            time.sleep(0.05)





    # The following code will build the firmware for the target.

    # In[ ]:


    try:
        get_ipython().run_cell_magic('bash', '-s "$PLATFORM" "$SS_VER"', 'cd ../../../hardware/victims/firmware/basic-passwdcheck\nmake PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2\n &> /tmp/tmp.txt')
    except:
        x=open("/tmp/tmp.txt").read(); print(x); raise OSError(x)



    # Finally, all that's left is to program the device, which can be done with the following line:

    # In[ ]:


    cw.program_target(scope, prog, "../../../hardware/victims/firmware/basic-passwdcheck/basic-passwdcheck-{}.hex".format(PLATFORM))


    # To make interacting with the hardware easier, let's define a function to attempt a password and return a power trace:

    # In[ ]:


    def cap_pass_trace(pass_guess):
        reset_target(scope)
        num_char = target.in_waiting()
        while num_char > 0:
            target.read(num_char, 10)
            time.sleep(0.01)
            num_char = target.in_waiting()

        scope.arm()
        target.write(pass_guess)
        ret = scope.capture()
        if ret:
            print('Timeout happened during acquisition')

        trace = scope.get_last_trace()
        return trace


    # We also don't need all of the default 5000 samples in the trace. 3000 is a good starting point for most targets:

    # In[ ]:


    scope.adc.samples = 3000



elif VERSION == 'SIMULATED':

    #!/usr/bin/env python
    # coding: utf-8

    # # Power Analysis for Password Bypass - SIMULATED Setup

    # ---
    # **THIS IS NOT THE COMPLETE TUTORIAL - see file with `(MAIN)` in the name.**
    #
    # ---

    # Sure you don't have hardware, but that doesn't mean we can't have fun! If you check the ChipWhisperer based lab (using hardware), you'll find that the capture function is defined like this:
    #
    #     def cap_pass_trace(pass_guess):
    #         ret = ""
    #         reset_target(scope)
    #         num_char = target.in_waiting()
    #         while num_char > 0:
    #             ret += target.read(num_char, 10)
    #             time.sleep(0.01)
    #             num_char = target.in_waiting()
    #
    #         scope.arm()
    #         target.write(pass_guess)
    #         ret = scope.capture()
    #         if ret:
    #             print('Timeout happened during acquisition')
    #
    #         trace = scope.get_last_trace()
    #         return trace
    #
    # This sends a password guess to the target device, and returns a power trace associated with the guess in question. So for example you could run:
    #
    #     cap_pass_trace("abcde\n")
    #
    # To get a power trace of `abcde`.
    #
    # Instead, we have a function that uses pre-recorded data. Run the following block and it should give you access to a function that uses pre-recorded data. While how you use the function is the same, note the following limitations:
    #
    # * Not every combination is stored in the system -- instead it stores similar power traces.
    # * 100 traces are stored for each guess, and it randomly returns one to still give you the effect of noise.
    #

    # In[ ]:



    #!/usr/bin/env python
    # coding: utf-8

    # In[ ]:


    import pickle
    import random
    import time

    traces_to_load = pickle.load(open("traces/lab2_1b_passwords_full.p", "rb"))

    def cap_pass_trace(pass_guess):
        if pass_guess.endswith("\n") is False:
            raise ValueError("Password guess must end with \\n")

        pass_guess = pass_guess.strip("\n")

        known_passwd = "h0px3"

        trylist = "abcdefghijklmnopqrstuvwxyz0123456789 \x00"

        if len(pass_guess) > 5:
            raise ValueError("Only guesses up to 5 chars recorded, sorry about that.")

        for a in pass_guess:
            if a not in trylist:
                raise ValueError("Part of guess (%c) not in recorded enumeration list (%s)"%(a, trylist))

        #Only recorded is correct passwords
        recorded_pw = ""
        for i in range(0, len(pass_guess)):
            if known_passwd[i] != pass_guess[i]:
                recorded_pw += " "
            else:
                recorded_pw += pass_guess[i]

        time.sleep(0.05)

        return traces_to_load[recorded_pw][random.randint(0, 99)]




    trace_test = cap_pass_trace("h\n")

    #Basic sanity check
    assert(len(trace_test) == 3000)
    print("✔️ OK to continue!")



bash: line 1: cd: ../../../hardware/victims/firmware/basic-passwdcheck: No such file or directory
make: *** No targets specified and no makefile found.  Stop.


INFO: Found ChipWhisperer😍
Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more


FileNotFoundError: [Errno 2] No such file or directory: '../../../hardware/victims/firmware/basic-passwdcheck/basic-passwdcheck-CWNANO.hex'

In [3]:
scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial)
scope.default_setup()


In [5]:
    try:
        target = cw.target(scope, target_type)
    except:
        print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
        print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
        scope = cw.scope(hw_location=(5, 16))
        target = cw.target(scope, target_type)


    print("INFO: Found ChipWhisperer😍")


INFO: Found ChipWhisperer😍


In [7]:
PLATFORM="CWNANO"



In [8]:
    if "STM" in PLATFORM or PLATFORM == "CWLITEARM" or PLATFORM == "CWNANO":
        prog = cw.programmers.STM32FProgrammer
    elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        prog = cw.programmers.XMEGAProgrammer
    elif "neorv32" in PLATFORM.lower():
        prog = cw.programmers.NEORV32Programmer
    elif PLATFORM == "CW308_SAM4S":
        prog = cw.programmers.SAM4SProgrammer
    else:
        prog = None


In [9]:
prog

chipwhisperer.capture.api.programmers.STM32FProgrammer

In [12]:
    import time
    time.sleep(0.05)
    scope.default_setup()

    if PLATFORM == "CW308_SAM4S":
        scope.io.target_pwr = 0
        time.sleep(0.2)
        scope.io.target_pwr = 1
        time.sleep(0.2)
    def reset_target(scope):
        if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
            scope.io.pdic = 'low'
            time.sleep(0.1)
            scope.io.pdic = 'high_z' #XMEGA doesn't like pdic driven high
            time.sleep(0.1) #xmega needs more startup time
        elif "neorv32" in PLATFORM.lower():
            raise IOError("Default iCE40 neorv32 build does not have external reset - reprogram device to reset")
        elif PLATFORM == "CW308_SAM4S":
            scope.io.nrst = 'low'
            time.sleep(0.25)
            scope.io.nrst = 'high_z'
            time.sleep(0.25)
        else:
            scope.io.nrst = 'low'
            time.sleep(0.05)
            scope.io.nrst = 'high_z'
            time.sleep(0.05)




In [14]:

    try:
        get_ipython().run_cell_magic('bash', '-s "$PLATFORM" "$SS_VER"', 'cd /home/init0/chipwhisperer/hardware/victims/firmware/basic-passwdcheck\nmake PLATFORM=$1 CRYPTO_TARGET=NONE SS_VER=$2\n &> /tmp/tmp.txt')
    except:
        x=open("/tmp/tmp.txt").read(); print(x); raise OSError(x)



SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
make[1]: '.dep' is up to date.
SS_VER set to SS_VER_2_1
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc (Arch Repository) 12.2.0
Copyright (C) 2022 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling:


In file included from .././hal/hal.h:22,
                 from basic-passwdcheck.c:19:
/usr/lib/gcc/arm-none-eabi/12.2.0/include/stdint.h:9:16: fatal error: stdint.h: No such file or directory
    9 | # include_next <stdint.h>
      |                ^~~~~~~~~~
compilation terminated.
make[1]: *** [.././Makefile.inc:494: objdir-CWNANO/basic-passwdcheck.o] Error 1
make: *** [.././Makefile.inc:337: all] Error 2


    basic-passwdcheck.c ...